## Set up google colab environment

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification')

In [2]:
from SGCNN_8_Utils import *
import scipy.io as sio

## Load Indian Pines Dataset - Source

In [3]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [4]:
data_source = uIndianPines['indian_pines_corrected']
ground_truth_source = gt_IndianPines['indian_pines_gt']

In [5]:
data_source.shape

(145, 145, 200)

In [6]:
ground_truth_source.shape

(145, 145)

# Load target dataset

In [7]:
uBotswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana.mat')
gt_Botswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana_gt.mat')

In [8]:
data_target = uBotswana['Botswana']
ground_truth_target = gt_Botswana['Botswana_gt']

In [9]:
data_target.shape

(1476, 256, 145)

In [10]:
ground_truth_target.shape

(1476, 256)

## Distrubution of samples for each class in Source

In [11]:
class_distribution_source = pd.DataFrame(np.unique(ground_truth_source, return_counts = True))
class_distribution_source = class_distribution_source.transpose()
class_distribution_source.columns = ['class','samples']
class_distribution_source

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [12]:
classes_source , counts_source = np.unique(ground_truth_source, return_counts = True)
classes_source = classes_source[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes_source

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

# Class distribution of samples in Target

In [13]:
class_distribution_target = pd.DataFrame(np.unique(ground_truth_target, return_counts = True))
class_distribution_target = class_distribution_target.transpose()
class_distribution_target.columns = ['class','samples']
class_distribution_target

,class,samples
0,0,374608
1,1,270
2,2,101
3,3,251
4,4,215
5,5,269
6,6,269
7,7,259
8,8,203
9,9,314


In [14]:
classes_target , counts_target = np.unique(ground_truth_target, return_counts = True)
classes_target = classes_target[1:] ## Dropping classes with small number of samples
classes_target

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with different overlap ratios and a percent of  samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [15]:
pretrain_results = pretrain_source_models(percentages = [60,70,80,90],
                                          classes = classes_source,
                                          cube_size = 20,
                                          overlap_ratios = [0.8,0.9,1],
                                          data = data_source,
                                          ground_truth = ground_truth_source,
                                          batch_size = 20,
                                          channels = 64,
                                          epochs = 100,
                                          Verbosity = 1,
                                          accuracies = [],
                                          learning_rate = 0.0001,
                                          source_dataset = 'indian_pines')

Streaming output truncated to the last 5000 lines.
118/118 [==============================] - 1s 12ms/step - loss: 2.5102 - categorical_accuracy: 0.8896 - val_loss: 3.1204 - val_categorical_accuracy: 0.6140

Epoch 00031: val_categorical_accuracy did not improve from 0.66561
Epoch 32/100
118/118 [==============================] - 1s 13ms/step - loss: 2.4990 - categorical_accuracy: 0.8823 - val_loss: 3.9943 - val_categorical_accuracy: 0.5535

Epoch 00032: val_categorical_accuracy did not improve from 0.66561
Epoch 33/100
118/118 [==============================] - 2s 13ms/step - loss: 2.4611 - categorical_accuracy: 0.8981 - val_loss: 6.4243 - val_categorical_accuracy: 0.3293

Epoch 00033: val_categorical_accuracy did not improve from 0.66561
Epoch 34/100
118/118 [==============================] - 1s 12ms/step - loss: 2.4599 - categorical_accuracy: 0.8991 - val_loss: 6.4339 - val_categorical_accuracy: 0.3115

Epoch 00034: val_categorical_accuracy did not improve from 0.66561
Epoch 35/100
1

In [16]:
pretrain_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,477,323,60,64.71
1,0.8,555,245,70,66.94
2,0.8,636,164,80,74.39
3,0.8,715,85,90,75.29
4,0.9,2348,1570,60,66.56
5,0.9,2740,1178,70,75.38
6,0.9,3132,786,80,75.95
7,0.9,3523,395,90,73.16
8,1.0,4740,3166,60,74.04
9,1.0,5530,2376,70,71.25


# Fine tune on botswana

In [17]:
transfer_results, confusion_matrixes = transfer_learning(percentages = [60,70,80,90],
                                                        source_dataset = 'indian_pines',
                                                        target_dataset = 'botswana',
                                                        data = data_target,
                                                        ground_truth = ground_truth_target,
                                                        classes = classes_target,
                                                        overlap_ratios = [0.8,0.9,1],
                                                        channels = 64,
                                                        cube_size = 20,
                                                        learning_rate = 0.0001,
                                                        epochs = 100,
                                                        batch_size = 20)

Streaming output truncated to the last 5000 lines.
batch_normalization_15 (BatchNo (None, 10, 10, 64)   256         conv_initial[0][0]               
__________________________________________________________________________________________________
re_lu_3 (ReLU)                  (None, 10, 10, 64)   0           batch_normalization_15[0][0]     
__________________________________________________________________________________________________
MaxPooling_0 (MaxPooling2D)     (None, 5, 5, 64)     0           re_lu_3[0][0]                    
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 5, 5, 64)     4160        MaxPooling_0[0][0]               
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 5, 5, 64)     256         conv2d_57[0][0]                  
__________________________________________________________

# Transfer Learning results on Botswana

In [18]:
transfer_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,251,175,60,69.14
1,0.8,293,133,70,61.65
2,0.8,337,89,80,76.40
3,0.8,378,48,90,58.33
4,0.9,914,615,60,74.31
5,0.9,1065,464,70,76.08
6,0.9,1219,310,80,94.19
7,0.9,1371,158,90,89.24
8,1.0,1880,1264,60,82.67
9,1.0,2195,949,70,94.63


# Classification accuracies per class for each model

In [19]:
for cm in confusion_matrixes:
  print(cm)

                1  2   3   4   5  ...  11  12  13  14  classfication_accuracies
0              12  0   0   0   2  ...   0   0   0   0                     85.71
1               0  0   0   0   6  ...   0   0   0   0                       0.0
2               0  0  14   0   0  ...   0   0   0   0                     100.0
3               0  0   1   3   0  ...   0   0   0   0                      25.0
4               3  0   0   0  20  ...   0   0   0   0                     83.33
5               0  0   0   0   1  ...   0   0   5   0                       0.0
6               3  0   0   0   0  ...   0   0   0   0                     76.92
7               0  0   0   0   0  ...   0   0   1   0                      80.0
8               0  0   0   0   0  ...   0   0   0   0                     100.0
9               0  0   0   0   0  ...   8   0   0   0                     27.27
10              0  0   3   0   0  ...  10   0   0   0                     76.92
11              0  0   0   0   0  ...   